In [1]:
import cv2
import os
import numpy as np

In [8]:
dataset_path='data/'
k=0
names={}
train_x=[]
train_y=[]
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        print(fx)
        face_data=np.load(dataset_path+fx)
        print(face_data.shape)
        c=0
        for i in face_data[:]:
            c+=1
            i=i.flatten()
            #print(i)
            #print(i.shape)
            train_x.append(i)
            train_y.append(k)
        names[k]=fx[0:len(fx)-4]
        k+=1
#train_data is a 2d matrix with 3000+1 columns where total features are 3000

Sarthak.npy
(100, 100, 100, 3)
Shaily.npy
(100, 100, 100, 3)


In [9]:
print(train_x)
print(train_y)

[array([15, 15, 15, ..., 18, 12, 11], dtype=uint8), array([15, 15, 16, ..., 16, 20, 21], dtype=uint8), array([17, 16, 18, ..., 32, 26, 19], dtype=uint8), array([16, 14, 15, ..., 24, 24, 24], dtype=uint8), array([28, 28, 29, ..., 23, 25, 20], dtype=uint8), array([121, 113, 101, ...,  18,  20,  18], dtype=uint8), array([112, 102,  97, ...,  22,  18,  12], dtype=uint8), array([128, 130, 128, ...,  25,  23,  26], dtype=uint8), array([56, 28, 25, ..., 50, 41, 29], dtype=uint8), array([91, 74, 71, ..., 48, 43, 34], dtype=uint8), array([47, 24, 19, ..., 56, 46, 37], dtype=uint8), array([36, 12,  7, ..., 33, 35, 32], dtype=uint8), array([144, 135, 140, ...,  25,  17,  17], dtype=uint8), array([135, 136, 142, ...,  17,  15,  14], dtype=uint8), array([137, 129, 135, ...,  20,  16,  19], dtype=uint8), array([126, 117, 125, ...,  19,  16,  14], dtype=uint8), array([124, 113, 114, ...,  55,  53,  44], dtype=uint8), array([113, 120, 115, ..., 112, 107, 105], dtype=uint8), array([133, 124, 122, ..., 

In [10]:
# converting list into numpy array
train_x=np.asarray(train_x)
print(train_x.shape)
train_y=np.asarray(train_y)
print(train_y.shape)

(200, 30000)
(200,)


In [11]:
print(names)

{0: 'Sarthak', 1: 'Shaily'}


In [12]:
def dis(X,Y):
    return np.sqrt(sum((X-Y)**2))

def KNN(x,y,query_x,k=5):
    val=[]
    m=x.shape[0]
    for i in range(m):
        d=dis(x[i],query_x)
        val.append((d,int(y[i])))
    val=sorted(val)
    val=np.array(val)
    val=val[:k,1]
    val=np.unique(val,return_counts=True)
    if(len(val[1])==2):
        if(val[1][0]==val[1][1]):
            if(k<100):
                return KNN(x,y,query_x,k+1)
    max_freq_index=val[1].argmax()
    return val[0][max_freq_index]

In [13]:
cap =  cv2.VideoCapture(0)  
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

skip=0
offset=10
face_data=[]
show_on_frame=[]

while True:
    ret,frame=cap.read()
   
    if ret==False:
        continue
    X=frame.shape[0]
    Y=frame.shape[1]
    
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    if skip%4==0:
        show_on_frame=[]
        for (x,y,w,h) in faces:
            Required_frame=frame[max(0,y-offset):min(Y,y+h+offset),max(0,x-offset):min(X,x+w+offset)]
            Required_frame=cv2.resize(Required_frame,(100,100))
            Required_frame=Required_frame.flatten()
            predicted_name=names[int(KNN(train_x,train_y,Required_frame))]
            show_on_frame.append((x,y,w,h,predicted_name))
            #         cv2.imshow("Detected Face",Required_frame)
            #         face_data.append(Required_frame)
            #         print(len(face_data))
    
    for (x,y,w,h,predicted_name) in show_on_frame:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
         # font 
        font = cv2.FONT_HERSHEY_SIMPLEX 
        # org
        org = (max(0,x-10),y)
        # fontScale
        fontScale = 1
        # Blue color in BGR 
        color = (255, 0, 0) 
        # Line thickness of 2 px 
        thickness = 2
   
        frame=cv2.putText(frame, predicted_name , org, cv2.FONT_HERSHEY_SIMPLEX,  
                   fontScale, color, thickness, cv2.LINE_AA)
    
    cv2.imshow("Scanning Your Face",frame)

    if cv2.waitKey(1) & 0xFF == ord('q') or len(face_data)>=1000:
        break
    skip+=1

cap.release()
cv2.destroyAllWindows()